# Week 11, Prep notebook

Last week we talked about how Jekyll sites work overall, and this week we'll focus on building visualizations to host on these pages with vega-lite and Altair in Python.

## 1. Include vega-lite plots directly from vega-editor

We can start with saving vega-lite code directly as json if we do our development in the vega-editor.  For a walk through of this see the file `3_direct_from_vega_editor.md` in the prep files (note to self: this is in the dev onlinecv locally at the moment).

**You must include the full URL to the dataset for this to work!**

## 2. Copy vega-lite code from other sources and save using Altair

For example, you may have developed some code on [Starboard](https://starboard.gg/) that you now want to copy into this notebook with Altair, and then eventually save to your Jekyll webpage.

We can follow the [Converting vega-lite to Altair instructions](https://altair-viz.github.io/user_guide/internals.html#converting-vega-lite-to-altair) to do this.

For example, from [one of our prep Starboard notebooks](https://starboard.gg/jnaiman/prep_notebook_week11_spring2022-nG3SEUx) we have the following vega-lite specification:

```
var myHist1 = 
{
  data: {"url": "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_spring2022/master/week10/data/mobility.csv"
  },
  mark: "bar",
  height: "300",
  width: "500",
  encoding: {
    "x": {"field": "State", "type": "nominal"},
    // NOTE: this won't work because "sum" assumes numerical data
    //"y": {"aggregate": "sum", "field": "State", "type": "nominal"} 
    //"y": {"aggregate": "count", "field": "State", "type": "nominal"} // might give an error/warning
    "y": {"aggregate": "count", "field": "State", "type": "quantitative"}
  }
};

var v = vegaEmbed('#firstHist', myHist1);
```

We can convert this to an Altair plot here with:

In [1]:
import altair as alt

In [5]:
chart1 = alt.Chart.from_dict({
  "data": {"url": "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_spring2022/master/week10/data/mobility.csv"
  },
  "mark": "bar",
  "height": 300,
  "width": 500,
  "encoding": {
    "x": {"field": "State", "type": "nominal"},
    # NOTE: this won't work because "sum" assumes numerical data
    #"y": {"aggregate": "sum", "field": "State", "type": "nominal"} 
    #"y": {"aggregate": "count", "field": "State", "type": "nominal"} // might give an error/warning
    "y": {"aggregate": "count", "field": "State", "type": "quantitative"}
  }
})

chart1

alt.Chart(...)

Note that I had to put in and take out some " marks -- this is because different vega-lite "hosting" services will be more/less lenient with these kinds of formatting issues.

Now, in principle, I could go to the Vega-Editor using the three `...` button at the upper right of this plot, OR I can just save with:

In [8]:
myJekyllDir = '/Users/jnaiman/online_cv_fall2022/'
chart1.properties(width='container').save(myJekyllDir+"assets/json/chart1.json")

## 3. Complications with copy-paste for more complex operations

So, not everything translates easily between how we've used vega-lite in Starboard before to here, in Jekyll.  In particular, selections in between plots can get a little tricky.  To do this, let's see about re-making our Mobility dashboard. 

Let's first do the plots one at a time.  The rectangle plot spec looked like:

```
var rectPlot1Spec = {
  // Data
  data: {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_spring2021/master/week08/data/mobility.csv"},
  // Marks
  mark:"rect",
  height:"400",
  // Encoding (note:error for encoding vs encodings)
  encoding:{
    //"x":{"field":"Student_teacher_ratio", "type":"quantitative"},
    "x":{"bin":{"maxbins":10}, "field":"Student_teacher_ratio", "type":"quantitative"},
    "y":{"field":"State","type":"ordinal"},
    "color":{"aggregate":"count", "type":"quantitative"} 
    // will show the number of records with a specific student/teacher ratio in a particular state
  }
  
};

var v = vegaEmbed('#rectPlot1',rectPlot1Spec);
```

Which is translated to:

In [9]:
chart1 = alt.Chart.from_dict({
  #// Data
  "data": {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_spring2021/master/week08/data/mobility.csv"},
  #// Marks
  "mark":"rect",
  "height":400,
  #// Encoding (note:error for encoding vs encodings)
  "encoding":{
    #//"x":{"field":"Student_teacher_ratio", "type":"quantitative"},
    "x":{"bin":{"maxbins":10}, "field":"Student_teacher_ratio", "type":"quantitative"},
    "y":{"field":"State","type":"ordinal"},
    "color":{"aggregate":"count", "type":"quantitative"} 
    #// will show the number of records with a specific student/teacher ratio in a particular state
  }
  
})
chart1

alt.Chart(...)

Note of course the changes in where " have been added and removed.

The second chart was the histogram of the mobility score, given by the specification:

```
var mobilityHistSpec = {
  // Data
  data: {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_spring2021/master/week08/data/mobility.csv"},
  // Mark
  mark: "bar",
  // Encoding
  encoding:{
    "x":{"field":"Mobility", "type":"quantitative", "bin":true, "axis":{"title":"Mobility Score"}},
    //"x":{"field":"Mobility", "type":"quantitative", "axis":{"title":"Mobility Score"}},
    "y":{"aggregate":"count","type":"quantitative", "axis":{"title":"Mobility Score Distribution"}}
  }
};

var v = vegaEmbed('#mobilityHist1',mobilityHistSpec);
```

This now turns into:

In [11]:
chart2 = alt.Chart.from_dict({
  #// Data
  "data": {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_spring2021/master/week08/data/mobility.csv"},
  #// Mark
  "mark": "bar",
  #// Encoding
  "encoding":{
    "x":{"field":"Mobility", "type":"quantitative", "bin":True, "axis":{"title":"Mobility Score"}},
    #//"x":{"field":"Mobility", "type":"quantitative", "axis":{"title":"Mobility Score"}},
    "y":{"aggregate":"count","type":"quantitative", "axis":{"title":"Mobility Score Distribution"}}
  }
})
chart2

alt.Chart(...)

Note here that we had to change the JS `true` to the Pythonic `True`.

The first thing we probably want to do is put these charts side-by-side.  We can do this with Altair's horizontal concatination function:

In [12]:
chart = alt.HConcatChart(hconcat=[chart1,chart2])
chart

alt.HConcatChart(...)